## 시퀀스-투-시퀀스

In [ ]:
# seq2seq는 크게 인코더와 디코더 두 개의 모듈로 구성
# 입력: 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만든다. (컨텍스트 벡터)
# 인코더: 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송
# 디코더: 번역

### 문자 레벨 기계 번역기 구현하기

In [2]:
# seq2seq -> 훈련 데이터로 병렬 코퍼스(두 개 이상의 언어가 병렬적으로 구성된 코퍼스)가 필요
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [3]:
lines = pd.read_csv('./fra-eng/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 197463


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]  # 6만개만 저장
lines.sample(10)


,src,tar
33710,"Yeah, you're right.","Ouais, vous avez raison."
777,Taste it.,Goûte-la.
43841,I'm your best friend.,Je suis votre meilleur ami.
5156,I work alone.,Je travaille seule.
22368,You're fortunate.,Vous êtes chanceuse.
18430,I am pretty sure.,Je suis relativement sûr.
41100,Do you need the keys?,T'as besoin des clefs ?
48391,Don't throw that away.,Ne le jetez pas !
5406,I've no idea.,Je n'ai aucune idée.
4446,Are you home?,Êtes-vous chez vous ?


In [5]:
# tar(프랑스어 데이터)는 <sos>와 <eos>를 넣어주어야 한다. 
# <sos> -> \t
# <eos> -> \n
lines.tar = lines.tar.apply(lambda x: '\t ' + x + ' \n')
lines.sample(10)


,src,tar
20969,Tom clearly lied.,\t Tom a manifestement menti. \n
31179,Keep your hands up.,\t Garde les mains en l'air ! \n
53345,We can't see anything.,\t Nous n'arrivons à rien voir. \n
26303,They'll like that.,\t Ils vont aimer cela. \n
33557,Which team is ours?,\t Quelle équipe est la nôtre ? \n
31621,She made me a cake.,\t Elle me concocta un gâteau. \n
57491,I went straight to bed.,\t Je suis allée directement au lit. \n
55643,Have you been drinking?,\t Avez-vous bu ? \n
22649,Are you ready now?,\t Es-tu prête maintenant ? \n
31969,The door blew shut.,\t La porte claqua dans un souffle. \n


In [6]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src:
    for char in line: # char: 한 문자
        src_vocab.add(char)
        
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [7]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 79
target 문장의 char 집합 : 105


In [8]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [9]:
# 각 문자에 인덱스를 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)


{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

In [13]:
# 인덱스가 부여된 문자 집합을 가지고 훈련 데이터에 정수 인코딩을 수행
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [14]:
# 프랑스어 데이터에 대해서 정수 인코딩을 수행
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2]]


In [15]:
# 디코더의 예측값과 비교하기 위한 실제값 정수 인코딩도 필요
# 실제값은 앞에 \t가 없으므로 제거해준 상태로 정수 인코딩이 필요
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2]]


In [16]:
# 문장 길이 맞추기
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

encoder_input = pad_sequences(
    encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(
    decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(
    decoder_target, maxlen=max_tar_len, padding='post')


source 문장의 최대 길이 : 23
target 문장의 최대 길이 : 76


In [18]:
# 모든 값에 대해서 원-핫 인코딩 수행
# 문자 단위 번역기므로 워드 임베딩은 별도로 사용되지 않음
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print(encoder_input[:5])

[[[[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  ...

  [[0. 1.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  ...

  [[0. 1.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]
   [1. 0.]
   ...
   [1. 0.]
   [1. 0.]
   [1. 0.]]

  [[1. 

In [19]:
# 교사 강요
# 훈련 과정에서 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣지 X
# 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 사용
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np


In [20]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태. -> 이 두가지가 디코더에 전달됨
encoder_states = [state_h, state_c]


In [21]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")


In [24]:
# 훈련 과정에서는 인코더 입력, 디코더 입력, 디코더의 실제값인 decoder_target도 필요
model.fit(x=[encoder_input, decoder_input], y=decoder_target,
          batch_size=64, epochs=40, validation_split=0.2)


Epoch 1/40


ValueError: in user code:

    File "c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\sksoh

In [25]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)


In [26]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs,
                      outputs=[decoder_outputs] + decoder_states)


In [27]:
# 인덱스로부터 단어 만들기
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())


## Word-Level 번역기 만들기

In [29]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [34]:
num_samples = 33000 # 개수 제한

In [35]:
# 전처리 함수들을 구현(구두점 등을 제거, 단어와 구분)
def to_ascii(s):
      # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [36]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :', preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))


전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [42]:
# 디코더 출력 시퀀스에 <sos>, <eos> 붙히기
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("./fra-eng/fra.txt", "r", encoding='UTF8') as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [43]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [44]:
# 단어 집합 생성, 정수 인코딩, 패딩 진행
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")


In [45]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 8)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [46]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(
    src_vocab_size, tar_vocab_size))


영어 단어 집합의 크기 : 4672, 프랑스어 단어 집합의 크기 : 8137


In [47]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [50]:
# 데이터 섞기
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [24498 26881 22400 ... 26554 12519 18904]


In [51]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]


In [53]:
# 테스트 데이터 분리
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


검증 데이터의 개수 : 3300


In [54]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 8)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 8)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


In [55]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model


In [56]:
embedding_dim = 64
hidden_units = 64


In [57]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)  # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb)  # 패딩 0은 연산에서 제외
# 상태값 리턴을 위해 return_state는 True
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(
    enc_masking)  # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c]  # 인코더의 은닉 상태와 셀 상태를 저장


In [58]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units)  # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs)  # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['acc'])


In [59]:
# 테스트 단계 인코더
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [60]:
def decode_sequence(input_seq):
      # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [61]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환


def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
          sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence


In [64]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

1/1 [==============================] - 0s 18ms/step
입력문장 : no one can get in . 
정답문장 : personne ne peut y entrer . 
번역문장 : attrapes malheureux partiale peuvent partiale
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
입력문장 : how big you are ! 
정답문장 : que vous etes grand ! 
번역문장 : attrapes savon remuez toyota savon laver estomac refl
--------------------------------------------------
1/1 [==============================] - 0s 17ms/step
입력문장 : tom s home . 
정답문장 : tom est a la maison . 
번역문장 : attrapes partiale payait payait mignonne photogr
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step
입력문장 : i made her a dress . 
정답문장 : je lui ai confectionne une robe . 
번역문장 : idolatrait pigez rome fixe dix astucieux envieuse 
--------------------------------------------------
1/1 [==============================] - 0s 19ms/step
입력문장 : they re in danger . 
정답문장 : ils sont en danger . 
번역문장 : at

## 어텐션 메커니즘

In [ ]:
# seq2seq 단점
# 1. 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생
# 2. 기울기 소실

# 어텐션의 아이디어
# 디코더에서 출력 단어를 예측하는 매 시점마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고한다.

In [ ]:
# 닷-프로덕트 어텐션
# 어텐션 스코어 -> 소프트맥스(어텐션 스코어) -> 인코더 은닉 상태 * 소프트맥스(어텐션 스코어) -> 합함 -> 어텐션 값
# 어텐션 값 + 디코더 은닉 상태(concentrate) -> 예측

## 바다나우 어텐션

In [ ]:
# 닷-프로턱트에서는 디코더의 t시점의 은닉 상태를 사용했지만 바다나우 어텐션에서는 디코더의 t-1시점의 은닉 상태를 사용
# 계산식을 통해서 어텐션 스코어 구함 -> 소프트맥스(어텐션 스코어)를 통해서 어텐션 분포를 구함 -> 각 인코더의 어텐션 가중치와 각 인코더의 은닉 상태를 가중합 -> 어텐션 값(인코더의 문맥을 포함한 컨텍스트 벡터)
# 컨텍스트 벡터와 임베딩된 단어 벡터를 연결하여 입력으로 사용

## 양방향 LSTM과 어텐션 메커니즘